In [ ]:
install.packages("BiocManager")
BiocManager::install("org.Hs.eg.db")
BiocManager::install("edgeR")
BiocManager::install("limma")
BiocManager::install("Glimma")

In [ ]:
# Load librarys
library("gplots")
library('edgeR')
library('limma')
library('Glimma')
library('ggplot2')
library('RColorBrewer')
library('org.Hs.eg.db')
library("dplyr")
library("gplots")

In [ ]:
# Explore control vs cases Data to kwnow what is the bestway  to process data
ctrl_vs_cases_transcrip_data <- read.csv('../input/end-als/end-als/transcriptomics-data/DESeq2/ctrl_vs_case.csv')

In [ ]:
# Getting ctrl vs case group
ctrl_vs_cases_group <- factor(ctrl_vs_cases_transcrip_data$CtrlVsCase_Classifier)
# Delete ctrl vs cases from original dataset 
ctrl_vs_cases_procces <- subset(ctrl_vs_cases_transcrip_data, select= -c(2))
# Transpose data to have id in columns
ctrl_vs_cases_procces <- t(ctrl_vs_cases_procces)
# Change the columns names for the ids
colnames(ctrl_vs_cases_procces) <- ctrl_vs_cases_procces[1, ]
# Delete  ids from the first row 
ctrl_vs_cases_procces <- ctrl_vs_cases_procces[-c(1), ]
# set rows names to add later to df
genes_names <- rownames(ctrl_vs_cases_procces)
ctrl_vs_cases_procces <- data.frame(ctrl_vs_cases_procces)
# changing the type data chart to numeric
ctrl_vs_cases_procces <- as.data.frame(sapply(ctrl_vs_cases_procces, as.numeric))
# Set row names genes to the new data colletion
rownames(ctrl_vs_cases_procces) <- genes_names

In [ ]:
# DGEList to run MDS
ctrl_vs_cases_procces_dglist <- DGEList(counts = as.matrix(ctrl_vs_cases_procces), group = ctrl_vs_cases_group)
#plot MDS 
plotMDS(ctrl_vs_cases_procces_dglist, col=as.numeric(ctrl_vs_cases_procces_dglist$samples$group))
legend("bottomleft", as.character(unique(ctrl_vs_cases_procces_dglist$samples$group)), col=1:3, pch=20)
#  CTRL VS CASES DESq2 Data MDS

In [ ]:
# Reading principal path L3 counts data to extrac txt files
l3_count_main_path <- "../input/end-als/end-als/transcriptomics-data/L3_counts"
# List of folder in L3_counts
l3_count_main_path_list <- list.files(path = l3_count_main_path)
# Declare variable just for concatenate txt files
all_transcriptomic_counts <- "Principal"
flag <- TRUE
for (folder in l3_count_main_path_list){
    # Get sub-folder path
    subfolder <- list.files(path=paste(l3_count_main_path, folder, sep='/'))
    # If the folder contains more than one file the is excluded
    if (length(subfolder) > 1){
        next
    }
    # Read last file this path contain the name of .txt file
    exon_txt_file <- list.files(path=paste(l3_count_main_path, folder, subfolder, sep='/'))
    # Concatenate all path to read csv
    complete_l3_count_path <- paste(l3_count_main_path, folder, subfolder, exon_txt_file, sep='/')
    
    # Reading exo.txt file
    l3_counts_per_id <- read.table(complete_l3_count_path, sep = "\t", header=TRUE)
    # Select genes columns and counts
    l3_counts_per_id <- subset(l3_counts_per_id, select=(c(1,7)))
    # Change the name of columns for genes for genes and counts per id
    colnames(l3_counts_per_id) <- c("genes", folder)
    # If is the first iterarion then enter into if and save file in all_transcript_count to by concacatenate later
    if(flag){
        flag<- FALSE
        all_transcriptomic_counts <- l3_counts_per_id
        next
    }
    # Join all count in single file
    all_transcriptomic_counts <- left_join(x = all_transcriptomic_counts, y = l3_counts_per_id, by = "genes", all.x = TRUE)
}
# Write csv file of raw counts
write.csv(all_transcriptomic_counts, "raw_L3counts_data.csv")

In [ ]:
# To performance MDS with color by case and ctrl is necesario get the groups from clinical data portal data
clinical_data_portal <- read.csv('../input/end-als/end-als/clinical-data/filtered-metadata/metadata/aals_dataportal_datatable.csv')
# Creating dataframe just with data that we need
clinical_data_portal_subset <- data.frame(clinical_data_portal$Participant_ID, clinical_data_portal$Sex,
                                         clinical_data_portal$Race, clinical_data_portal$Subject.Group, 
                                         clinical_data_portal$Site.of.Onset, clinical_data_portal$NEFH..SMI32.,
                                         clinical_data_portal$ISL1, clinical_data_portal$`NKX6.1`, 
                                         clinical_data_portal$TUBB3..TuJ1., clinical_data_portal$s100b)
# Change col names
colnames(clinical_data_portal_subset) <- c("Participant_ID", "Sex", "Race", "SubjectGroup", "SiteOfOnset", "NEFH",
                                          "ISL1", "NKX6", "TUBB3", "s100b")
# Transpose data to join data portal and raw counts data
all_transcriptomic_counts_transpose <- t(all_transcriptomic_counts)
# Change the columns names for the ids
colnames(all_transcriptomic_counts_transpose) <- all_transcriptomic_counts_transpose[1, ]
# Delete  ids from the first row 
all_transcriptomic_counts_transpose <- data.frame(all_transcriptomic_counts_transpose[-c(1), ])
# set participant Id as columns
all_transcriptomic_counts_transpose$Participant_ID <- rownames(all_transcriptomic_counts_transpose)
# Join raw counts and data portal 
all_transcriptomic_counts_and_data_portal <- left_join(x = all_transcriptomic_counts_transpose, 
                                       y = clinical_data_portal_subset, by = "Participant_ID", all.x = TRUE)

# Save transcripts groups to use latter in Difererntial expression analysis
transcripts_groups <- all_transcriptomic_counts_and_data_portal[c("Participant_ID", "Sex", "Race", 
                                                                 "SubjectGroup", "SiteOfOnset", "NEFH",
                                                                 "ISL1", "NKX6", "TUBB3", "s100b")]

transcripts_groups$SubjectGroup[which(transcripts_groups$SubjectGroup != "Healthy Control")]  <- "ALS"
# Delete columns from raw counts
all_transcriptomic_counts_and_data_portal <- subset(all_transcriptomic_counts_and_data_portal, select=-c( Sex, Race, 
                                                                 SubjectGroup, SiteOfOnset, NEFH,
                                                                 ISL1, NKX6, TUBB3, s100b))

In [ ]:
# report process to transpose data
# Transpose data to have id in columns
all_transcriptomic_counts_and_data_portal <- t(all_transcriptomic_counts_and_data_portal)
# Change the columns names for  ids patinets
colnames(all_transcriptomic_counts_and_data_portal) <- all_transcriptomic_counts_and_data_portal[60676, ]
# Delete  ids from the last row 
all_transcriptomic_counts_and_data_portal <- all_transcriptomic_counts_and_data_portal[-c(60676), ]
# Convert matrix to data frame
all_transcriptomic_counts_and_data_portal <- data.frame(all_transcriptomic_counts_and_data_portal)
# Set genes ids
genes_id <- rownames(all_transcriptomic_counts_and_data_portal)
# # changing the type data chart to numeric
all_transcriptomic_counts_and_data_portal <- as.data.frame(lapply(data.frame(lapply(all_transcriptomic_counts_and_data_portal, as.character), stringsAsFactors=FALSE), as.numeric))
# Set row names genes to the new data colletion
rownames(all_transcriptomic_counts_and_data_portal) <- genes_id

In [ ]:
# change group type factor to character
transcripts_groups$Sex <-  as.character(transcripts_groups$Sex)
transcripts_groups$SubjectGroup <-  as.character(transcripts_groups$SubjectGroup)
transcripts_groups$SiteOfOnset <-  as.character(transcripts_groups$SiteOfOnset)
transcripts_groups$Race <- as.character(transcripts_groups$Race)

In [ ]:
# Filter by markers
# it'is neccesary create boolean list of each marker to filter all missings values
# NEFH marker filter
NEFH_boolean_list <- !is.na(transcripts_groups$NEFH)
all_transcriptomic_counts_and_data_portal <- all_transcriptomic_counts_and_data_portal [NEFH_boolean_list]
transcripts_groups <- transcripts_groups[NEFH_boolean_list, ]

# ISL1 marker filter
ISL1_boolean_list <- !is.na(transcripts_groups$ISL1)
all_transcriptomic_counts_and_data_portal <- all_transcriptomic_counts_and_data_portal [ISL1_boolean_list]
transcripts_groups <- transcripts_groups[ISL1_boolean_list, ]

# NKX6 marker filter
NKX6_boolean_list <- !is.na(transcripts_groups$NKX6)
all_transcriptomic_counts_and_data_portal <- all_transcriptomic_counts_and_data_portal [NKX6_boolean_list]
transcripts_groups <- transcripts_groups[NKX6_boolean_list, ]

# TUBB3 marker filter
TUBB3_boolean_list <- !is.na(transcripts_groups$TUBB3)
all_transcriptomic_counts_and_data_portal <- all_transcriptomic_counts_and_data_portal [TUBB3_boolean_list]
transcripts_groups <- transcripts_groups[TUBB3_boolean_list, ]

# s100b marker filter
s100b_boolean_list <- !is.na(transcripts_groups$s100b)
all_transcriptomic_counts_and_data_portal <- all_transcriptomic_counts_and_data_portal [s100b_boolean_list]
transcripts_groups <- transcripts_groups[s100b_boolean_list, ]

In [ ]:
dgelist_edgeR_normalize <- DGEList(counts = as.matrix(all_transcriptomic_counts_and_data_portal), group = transcripts_groups$SubjectGroup)
# Filter data by cpm
keep_data<- filterByExpr(dgelist_edgeR_normalize)
dgelist_edgeR_normalize<- dgelist_edgeR_normalize[keep_data, , keep.lib.sizes=FALSE]
# Normalize Data
edgeR_normalize_data <- calcNormFactors(dgelist_edgeR_normalize)
#edgeR_normalize_data <- logcpm <- cpm(edgeR_normalize_data, log=TRUE)
dgelist_edgeR_normalize <- DGEList(counts = as.matrix(edgeR_normalize_data), group = transcripts_groups$SubjectGroup)

In [ ]:
# plot MDS
plotMDS(dgelist_edgeR_normalize, col=as.numeric(dgelist_edgeR_normalize$samples$group))
legend("bottomleft", as.character(unique(dgelist_edgeR_normalize$samples$group)), col=1:3, pch=20)
# EdgeR normalization and plot MDS

In [ ]:
# Calculate count per million and keep samples
cpm_quantile_transform <- cpm(all_transcriptomic_counts_and_data_portal)
# thresh_cpm to filter data
thresh_cpm <- cpm_quantile_transform > 0.5
keep_quantile_data <- rowSums(thresh_cpm) >= 2
counts.keep_quantile_data<- all_transcriptomic_counts_and_data_portal[keep_quantile_data,]
# Quantile normalization
quantile_normalization_data <-normalizeQuantiles(counts.keep_quantile_data, ties=TRUE)
# log transformation
log_quantile_norm_data <- log1p(quantile_normalization_data)
#created dglist to make MDS
dgelist_quantile_data <- DGEList(counts = as.matrix(log_quantile_norm_data), group = transcripts_groups$SubjectGroup)

In [ ]:
# plot MDS
plotMDS(dgelist_quantile_data, col=as.numeric(dgelist_quantile_data$samples$group))
legend("bottomleft", as.character(unique(dgelist_quantile_data$samples$group)), col=1:3, pch=20)
# Quantile normalization data filter after removing the low count genes MDS

In [ ]:
# Diferential expression analisis
NKX <- transcripts_groups$NEFH
NEFH <- transcripts_groups$ISL1
TUBB3 <- transcripts_groups$NKX6
ISL1 <- transcripts_groups$TUBB3
s100b <- transcripts_groups$s100b
sex <- transcripts_groups$Sex
race <- transcripts_groups$Race
cases_vs_control <- factor(transcripts_groups$SubjectGroup)

In [ ]:
design_matrix_cases_control <- model.matrix(~0 + cases_vs_control + NKX + NEFH + TUBB3 + ISL1 + s100b + race + sex )

colnames(design_matrix_cases_control) <- c('ALS', 'CTRL','NKX', 'NEFH', 'TUBB3', 'ISL1', 's100b',
                     'race1', 'race2',
                        'race3', 'rece4','rece5', 'sex') 


# fit data
fit_case_vs_ctrl <- lmFit(as.matrix(log_quantile_norm_data), design_matrix_cases_control)
# contrast case vs control data
cont.matrix_cases_control <- makeContrasts(ALS-CTRL, levels = design_matrix_cases_control)
fit_case_vs_ctrl <- contrasts.fit(fit_case_vs_ctrl, cont.matrix_cases_control)
fit_case_vs_ctrl <- eBayes(fit_case_vs_ctrl)

#cluster1 vs all
DEP_SIG_ALS_case_vs_control <- topTable(fit_case_vs_ctrl, adjust = "BH", coef = 1,
                                            number = length(fit_case_vs_ctrl$coefficients)) 
# # Filter by  p.adjust.val
DEP_SIG_ALS_case_vs_control <- DEP_SIG_ALS_case_vs_control[ DEP_SIG_ALS_case_vs_control$adj.P.Val < 0.05, ]
DEP_SIG_ALS_case_vs_control

In [ ]:
# Filter data jus for cases 
all_transcriptomic_counts_and_data_portal <- all_transcriptomic_counts_and_data_portal[transcripts_groups$SubjectGroup == 'ALS']
transcripts_groups <- transcripts_groups[transcripts_groups$SubjectGroup == 'ALS', ]

In [ ]:
# Calculate count per million and keep samples
cpm_quantile_transform <- cpm(all_transcriptomic_counts_and_data_portal)
# thresh_cpm to filter data
thresh_cpm <- cpm_quantile_transform > 0.5
keep_quantile_data <- rowSums(thresh_cpm) >= 2
counts.keep_quantile_data<- all_transcriptomic_counts_and_data_portal[keep_quantile_data,]
# Quantile normalization
quantile_normalization_data <-normalizeQuantiles(counts.keep_quantile_data, ties=TRUE)
# log transformation
log_quantile_norm_data <- log1p(quantile_normalization_data)
#created dglist to make MDS
dgelist_quantile_data <- DGEList(counts = as.matrix(log_quantile_norm_data), group = transcripts_groups$SubjectGroup)

In [ ]:
# plot MDS jus for Cases
plotMDS(dgelist_quantile_data, col=as.numeric(dgelist_quantile_data$samples$group))
legend("bottomleft", as.character(unique(dgelist_quantile_data$samples$group)), col=1:3, pch=20)

In [ ]:
# remove bath effect by markers
quantile_data_removeBatchEffects <- removeBatchEffect(log_quantile_norm_data, covariates=transcripts_groups[c("NEFH", "ISL1", "NKX6", "TUBB3", "s100b")])

In [ ]:
# Plot MDS After generating an MDS plot, we did not observe any more heterogeneity in the data.
plotMDS(quantile_data_removeBatchEffects)

In [ ]:
heatmap_corrected_data <- heatmap.2(as.matrix(quantile_data_removeBatchEffects), dendrogram='both', distfun = function(x) dist(x, method="euclidean"))

In [ ]:
hc <- as.hclust( heatmap_corrected_data$colDendrogram )
clusters <- cutree(hc, k=2 )
# clusters_corrected_data <
transcripts_groups$clusters <- clusters

In [ ]:
# Diferential expression analisis for cases
# Declare covariantes
NKX <- transcripts_groups$NEFH
NEFH <- transcripts_groups$ISL1
TUBB3 <- transcripts_groups$NKX6
ISL1 <- transcripts_groups$TUBB3
s100b <- transcripts_groups$s100b
sex <- transcripts_groups$Sex
race <- transcripts_groups$Race
clusters <- factor(transcripts_groups$clusters)

In [ ]:
design_matrix <- model.matrix(~0 + clusters + NKX + NEFH + TUBB3 + ISL1 + s100b + race + sex )

colnames(design_matrix) <- c('clusters_one1', 'clusters_one2','NKX', 'NEFH', 'TUBB3', 'ISL1', 's100b',
                     'race1', 'race2',
                        'race3', 'rece4', 'sex') 


# fit data
fit_case_clusters <- lmFit(as.matrix(log_quantile_norm_data), design_matrix)
# contrast case vs control data
cont.matrix_cluster <- makeContrasts(clusters_one1-clusters_one2, levels = design_matrix)
fit_case_clusters <- contrasts.fit(fit_case_clusters, cont.matrix_cluster)
fit_case_clusters <- eBayes(fit_case_clusters)

#cluster1 vs all
DEP_SIG_ALS_cluster1_vs_cluster2 <- topTable(fit_case_clusters, adjust = "BH", coef = 1,
                                           number = length(fit_case_clusters$coefficients)) 
# Filter by  p.adjust.val
DEP_SIG_ALS_cluster1_vs_cluster2 <- DEP_SIG_ALS_cluster1_vs_cluster2[ DEP_SIG_ALS_cluster1_vs_cluster2$adj.P.Val < 0.05, ]
DEP_SIG_ALS_cluster1_vs_cluster2
write.csv(DEP_SIG_ALS_cluster1_vs_cluster2, 'cluster1_vs_cluster2_corrected.csv') # Cluster1VsCluster2